# **Deep Learning Methods for Missing Value Imputation**

Deep learning techniques, particularly **neural networks** such as **autoencoders**, provide an effective way to impute missing values in complex datasets. These methods are especially beneficial when data exhibits **non-linear relationships** that traditional imputation techniques struggle to handle.

---

## **Understanding Autoencoders for Imputation**

### **What is an Autoencoder?**
An **autoencoder** is a type of neural network designed to **reconstruct its input**. It consists of two main components:  
- **Encoder**: Compresses the input into a **latent-space representation**.  
- **Decoder**: Reconstructs the original input from this compressed representation.  

This architecture allows autoencoders to learn a meaningful representation of data, making them useful for missing value imputation.

---

## **How Autoencoders Work for Imputation**
1. **Training with Incomplete Data**  
   - The model is trained with **incomplete inputs**, allowing it to learn to **ignore missing values**.  
   
2. **Predicting Missing Values**  
   - The autoencoder minimizes the **reconstruction error** for the known data, enabling it to **predict missing values** accurately.  

3. **Learning Robust Representations**  
   - By capturing the underlying patterns, autoencoders **generate reasonable approximations** for missing values.

---

## **Advantages of Using Autoencoders**
**Captures Complex Patterns**: Can model **non-linear relationships** in data.  
**Scalability**: Efficiently handles **large datasets**.  
**Flexibility**: Works with **different data types** (images, text, time-series, tabular data).  

---

## **Key Considerations for Implementation**
- **Data Preprocessing**: Inputs should be **normalized or standardized** before training.  
- **Network Architecture**: The **number and type of layers** should match the complexity of the dataset.  
- **Training Techniques**: Methods like **dropout** and **noise addition** can improve imputation performance.  

---

## **Example Use Cases**
**Image Data** → Reconstructs missing pixels or repairs corrupted images.  
**Time-Series Data** → Fills in missing values in sequential data (e.g., stock prices, weather patterns).  
**Tabular Data** → Handles missing values in structured datasets used for machine learning.  

---

### **Summary**
Autoencoders provide a **powerful deep learning-based approach** for imputing missing values by **learning robust data representations**. Their ability to **capture complex relationships** makes them ideal for handling large and diverse datasets across various domains.



In [2]:
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Load the Titanic dataset
df_titanic = sns.load_dataset('titanic')

# Selecting relevant features for simplicity
df_titanic = df_titanic[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]

# Preprocessing
# Separate features and target
X = df_titanic.drop('survived', axis=1)
y = df_titanic['survived']

# Handling missing values and categorical variables
numeric_features = ['age', 'fare', 'sibsp', 'parch']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())])

categorical_features = ['pclass', 'sex', 'embarked']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# ColumnTransformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Preprocessing the dataset
X_preprocessed = preprocessor.fit_transform(X)

# Splitting the dataset (we'll use the train set to train the autoencoder)
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Define the autoencoder architecture
input_dim = X_train.shape[1]
encoding_dim = 32

input_layer = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_layer)
decoded = Dense(input_dim, activation='sigmoid')(encoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=50, batch_size=256, shuffle=True, validation_split=0.2)

# Using the autoencoder for imputation on test set
X_test_imputed = autoencoder.predict(X_test)

# Note: Transforming imputed data back to original feature space is complex and requires reversing the preprocessing steps.
# This is often not straightforward, especially for one-hot encoded features.


Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - loss: 0.2452 - val_loss: 0.2438
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.2401 - val_loss: 0.2392
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.2359 - val_loss: 0.2347
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.2307 - val_loss: 0.2302
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.2279 - val_loss: 0.2259
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.2225 - val_loss: 0.2216
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.2183 - val_loss: 0.2174
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.2141 - val_loss: 0.2132
Epoch 9/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.2101 - val_loss: 0.2090
Epoch 10/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.2059 - val_loss: 0.2049
Epoch 11/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.2016 - val_loss: 0.2007
Epoch 12/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.1978 - val_loss: 0.1965
E